## Option 1: Heroes of Pymoli

In [5]:
import pandas as pd
import os
import numpy as np

df = pd.read_csv(os.path.join("Resources","purchase_data.csv"))

---------
### 1. Player Count
* Total Number of Players

In [6]:
pd.DataFrame({
    "Total Players": [len(df.SN.unique())]
})

,Total Players
0,576


## Purchasing Analysis (Total)
---

* Run basic calculations to obtain number of unique items, average price, etc.

In [8]:
unique_items = len(df['Item ID'].unique())
average_price = f"${df.Price.mean():.2f}"
number_of_purchases = len(df['Purchase ID'].unique())
total_revenue = f"${df.Price.sum():,.2f}"

* Create a summary data frame to hold the results

In [9]:
summary_df = pd.DataFrame({
    "Number of Unique Items":[unique_items],
    "Average Price": [average_price],
    "Number of Purchases": [number_of_purchases],
    "Total Revenue": [total_revenue]
})

* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [10]:
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


---
### 3. Gender Demographics
---
* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [11]:
df_gender = df.drop_duplicates(subset="SN").groupby('Gender').count().SN.to_frame().rename(columns={'SN':'Total Count'})
total = df_gender['Total Count'].sum()
df_gender['Percentage of Players'] = df_gender.apply(lambda x: f"{x['Total Count']/total*100:.2f}%", axis=1)
df_gender.sort_values(by="Total Count",ascending=False, inplace=True)
df_gender.rename_axis(None, axis=0, inplace=True)
df_gender

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


---
### 4. Purchasing Analysis (Gender)
---
- The below each broken by gender
    - Purchase Count
    - Average Purchase Price
    - Total Purchase Value
    - Average Purchase Total per Person by Gender

In [12]:
genderGrp = df.groupby('Gender')
pd.DataFrame({
    'Purchase Count': genderGrp.count()['Purchase ID'],
    'Average Purchase Price': genderGrp.mean().Price,
    'Total Purchase Value': genderGrp.sum().Price,
    'Avg Total Purchase per Person': genderGrp.sum().Price/df_gender['Total Count']
}).style.format({
    'Average Purchase Price': '${:.2f}',
    'Total Purchase Value': '${:,.2f}',
    'Avg Total Purchase per Person': '${:.2f}'
})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


---
### 5. Age Demographics
* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Optional: round the percentage column to two decimal points
* Display Age Demographics Table
---

In [15]:
bins = pd.IntervalIndex.from_tuples([(0,9),(9,14),(14,19),(19,24),(24,29),(29,34),(34,39),(39,50)])
df['bins'] = pd.cut(df.Age,bins=bins)
ageGrp = df.drop_duplicates('SN').groupby('bins')
total_count = ageGrp.count().SN
perc_players = total_count / total_count.sum()*100
age_demographics = pd.DataFrame({
    'Total Count': total_count,
    'Percentage of Players': perc_players.apply(lambda x: f"{x:.2f}%"),
    'Age Ranges': ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
}).set_index('Age Ranges')
age_demographics

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


---
## Purchasing Analysis (Age)
---
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame
---
* The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
    * Purchase Count
    * Average Purchase Price
    * Total Purchase Value
    * Average Purchase Total per Person by Age Group

In [18]:
bins = pd.IntervalIndex.from_tuples([(0,9),(9,14),(14,19),(19,24),(24,29),(29,34),(34,39),(39,50)])
df['bins'] = pd.cut(df.Age,bins=bins)
ageGrp = df.groupby('bins')
person_count = df.drop_duplicates('SN').groupby('bins').count().SN

df_age_bins = pd.DataFrame({
    'Purchase Count': ageGrp.count().SN,
    'Average Purchase Price': ageGrp.mean().Price.apply(lambda x: f"${x:.2f}"),
    'Total Purchase Value': ageGrp.sum().Price,
    'Age Ranges': ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
})
df_age_bins['Avg Total Purchase per Person'] = df_age_bins['Total Purchase Value']/person_count
df_age_bins.set_index('Age Ranges').style.format({
    'Total Purchase Value': '${:,.2f}',
    'Avg Total Purchase per Person': '${:.2f}'
})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


-------------
### 6. Top Spenders
* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
    * SN
    * Purchase Count
    * Average Purchase Price
    * Total Purchase Value

In [7]:
sn_group = df.groupby(by="SN")
top5 = sn_group['Price'].sum().nlargest(5).index
top5

df_top5 = pd.DataFrame(
    [],
    columns = ["SN","Purchase Count","Average Purchase Price","Total Purchase Value"]
)
df_top5

for index, user in enumerate(top5):
    df_top5.loc[index+1,"SN"] = user
    df_top5.loc[index+1,"Purchase Count"] = sn_group.get_group(user)["Purchase ID"].count()
    df_top5.loc[index+1,"Average Purchase Price"] = sn_group.get_group(user)["Price"].mean()
    df_top5.loc[index+1,"Total Purchase Value"] = sn_group.get_group(user)["Price"].sum()
    
df_top5.style.format({"Average Purchase Price":"${:.2f}","Total Purchase Value":"${:.2f}"})

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
1,Lisosia93,5,$3.79,$18.96
2,Idastidru52,4,$3.86,$15.45
3,Chamjask73,3,$4.61,$13.83
4,Iral74,4,$3.40,$13.62
5,Iskadarya95,3,$4.37,$13.10


--------
### 7. Most Popular Items
* Identify the 5 most popular items by purchase count, then list (in a table):
    * Item ID
    * Item Name
    * Purchase Count
    * Item Price
    * Total Purchase Value

In [8]:
item_group = df.groupby(by="Item ID")
top5_items = df.groupby(by="Item ID").count().sort_values(by="Purchase ID",ascending=False).head().reset_index()['Item ID']

df_top5_items = pd.DataFrame(
    data = [],
    columns = ['Item ID','Item Name','Purchase Count','Item Price','Total Purchase Value']
)  

for index, item in enumerate(top5_items):
    df_top5_items.loc[index+1, 'Item ID'] = item
    df_top5_items.loc[index+1, 'Item Name'] = item_group.get_group(item).reset_index().loc[0,'Item Name'] 
    df_top5_items.loc[index+1, 'Purchase Count'] = item_group.get_group(item).count()[0]
    df_top5_items.loc[index+1, 'Item Price'] = ", ".join([f"${x}" for x in item_group.get_group(item).reset_index()['Price'].unique()])
    df_top5_items.loc[index+1, 'Total Purchase Value'] = item_group.get_group(item).reset_index()['Price'].sum()
    
df_top5_items.style.format({"Total Purchase Value": "${:.2f}"})

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
1,92,Final Critic,13,"$4.88, $4.19",$59.99
2,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
3,145,Fiery Glass Crusader,9,$4.58,$41.22
4,132,Persuasion,9,"$3.19, $3.33",$28.99
5,108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


---------
### 8. Most Profitable Items
* Sort the above table by total purchase value in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the data frame

In [9]:
df_profit = df.groupby(by="Item ID").sum()['Price'].reset_index().rename(columns={"Price":"Total Purchase Value"}).sort_values(by="Total Purchase Value",ascending=False).set_index("Item ID")

df_profit = df_profit.merge(df[["Item ID","Item Name"]].drop_duplicates(), how='inner', on="Item ID")
df_profit = df_profit.merge(item_group.count().rename(columns={'SN':'Purchase Count'})[['Purchase Count']], how="left", on="Item ID")

df_profit["Item Price"] = [''] * df_profit['Item ID'].size

for index, item in enumerate(df_profit['Item ID']):
    df_profit.loc[index,"Item Price"] = ", ".join(f"${x:.2f}" for x in item_group.get_group(item).reset_index()['Price'].unique())

df_profit.head(10).style.format({"Total Purchase Value": "${:.2f}"})

,Item ID,Total Purchase Value,Item Name,Purchase Count,Item Price
0,92,$59.99,Final Critic,13,"$4.88, $4.19"
1,178,$50.76,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23
2,82,$44.10,Nirvana,9,$4.90
3,145,$41.22,Fiery Glass Crusader,9,$4.58
4,103,$34.80,Singed Scalpel,8,$4.35
5,59,$33.84,"Lightning, Etcher of the King",8,$4.23
6,108,$31.77,"Extraction, Quickblade Of Trembling Hands",9,$3.53
7,78,$30.80,"Glimmer, Ender of the Moon",7,$4.40
8,72,$30.16,Winter's Bite,8,$3.77
9,132,$28.99,Persuasion,9,"$3.19, $3.33"


### Final Considerations
##### Three (3) observable trends based on the data.
    1. Males make up the majority of the target market
    2. Age 20-24 has the most purchases as well as makes up the most gross profit.  Only one purchaser over age 45.
    3. Even the the top spenders didn't spend more than $20.